In [24]:
import numpy as np
import pandas as pd
from scipy.stats import gamma
import dask.array as da
from dask import delayed
import dask
from concurrent.futures import ProcessPoolExecutor
import gc
import time

start_time = time.time()

mean = 1
variance = 4
k = mean ** 2 / variance
theta = variance / mean

random_values = gamma.rvs(k, scale=theta, size=1000000)

percentile_99_5 = np.percentile(random_values, 99.5)

rating = ['A', 'BBB', 'BB', 'B', 'CCC']
p = np.array([0.06, 0.20, 1.25, 6.25, 17.50])
w = np.array([1.011, 0.836, 0.602, 0.415, 0.295])

rating_table = pd.DataFrame({
    'Rating': rating,
    'p_mean': p,
    'w': w
})

print('Table of associated values:')
print(rating_table)

E_LGD = 0.50

rating_table['mu_x'] = E_LGD * rating_table['p_mean'] * (1 + rating_table['w'] * (percentile_99_5 - 1))

print('Table with mu_x values:')
print(rating_table)

sigma = 2
lambda_ = 0.5
eta = 0.25
sigma2 = sigma ** 2

rating_table['Beta'] = (1 / (2 * lambda_)) * (lambda_ ** 2 + eta ** 2) * (
        (1 / sigma2) * (1 + (sigma2 - 1) / percentile_99_5) *
        (percentile_99_5 + (1 - rating_table['w']) / rating_table['w']) - 1)

n_values = [5000, 2000, 1000, 500, 200, 100]
column_names = ['VaR5000', 'VaR2000', 'VaR1000', 'VaR500', 'VaR200', 'VaR100']

for i, n in enumerate(n_values):
    rating_table[column_names[i]] = rating_table['mu_x'] + (rating_table['Beta'] / n) * 100

rating_table = rating_table.drop(columns=['Beta'])

rating_table.to_excel('/users/lorenzovancadsand/downloads/granularity_add-on.xlsx', index=False)

dask_data = da.from_array(random_values, chunks=100000)

@delayed
def compute_percentile(data):
    return np.percentile(data, 99.5)

result = compute_percentile(dask_data)

percentile_99_5_parallel = result.compute()
print(f'99° Percentile: {percentile_99_5_parallel}')


import random
from joblib import Parallel, delayed
from tqdm import tqdm
file_path = '/Users/lorenzovancadsand/Downloads/portafoglio_Asia.csv'
portfolio = pd.read_csv(file_path)

print('Portfolio simulations:')
print(portfolio)

lambda_ = 0.5
eta = 0.25

num_bonds = len(portfolio)

LGD_j = np.random.gamma(shape=lambda_, scale=eta, size=num_bonds)

portfolio['LGD_j'] = LGD_j
portfolio = portfolio.round(3)

regions = ['NORTH_AM', 'EUROPE', 'ASIA', 'AFRICA', 'LATIN_AM', 'REST_WORLD']
associated_regions = portfolio['Regione']

years = [2025, 2030, 2035, 2040, 2045, 2050]

print('Updated portfolio with LGD_j:')
print(portfolio)

filename_shock = '/Users/lorenzovancadsand/Downloads/Shock_Asia(Foglio1).csv'
shock_data = pd.read_csv(filename_shock)

print(shock_data.head())

shock_data = shock_data.round(3)

print(shock_data.head())

# Group and calculate the maximum shock for each combination of REGION, MODEL, SCENARIO
max_shock_by_region = shock_data.groupby(['REGION', 'MODEL', 'SCENARIO'], as_index=False).agg({
    '2025': 'max',
    '2030': 'max',
    '2035': 'max',
    '2040': 'max',
    '2045': 'max',
    '2050': 'max'
})

max_shock_by_region.rename(columns={
    '2025': 'Max_Shock_2025',
    '2030': 'Max_Shock_2030',
    '2035': 'Max_Shock_2035',
    '2040': 'Max_Shock_2040',
    '2045': 'Max_Shock_2045',
    '2050': 'Max_Shock_2050'
}, inplace=True)

print(max_shock_by_region.head(10))

print(shock_data)

sectors_data = {
    'primary_fossil': [
        'Primary Energy|Coal',
        'Primary Energy|Gas',
        'Primary Energy|Oil'
    ],
    'secondary_fossil': [
        'Secondary Energy|Electricity|Coal',
        'Secondary Energy|Electricity|Gas',
        'Secondary Energy|Electricity|Oil'
    ],
    'secondary_renewable': [
        'Secondary Energy|Electricity|Biomass',
        'Secondary Energy|Electricity|Geothermal',
        'Secondary Energy|Electricity|Solar|CSP',
        'Secondary Energy|Electricity|Nuclear',
        'Secondary Energy|Electricity|Solar|PV',
        'Secondary Energy|Electricity|Wind',
        'Secondary Energy|Electricity|Hydro'
    ]
}

print("Primary Fossil Sectors:")
print(sectors_data['primary_fossil'])

print("\nSecondary Renewable Sectors:")
print(sectors_data['secondary_renewable'])

model = 'REMIND'
scenario = 'LIMITSOilIndependence'

shock_data = shock_data[(shock_data['MODEL'] == model) & (shock_data['SCENARIO'] == scenario)]



num_simulations = 1000
max_subsectors_per_sector = 3

sectors_data = {
    'primary_fossil': ['Primary Energy|Coal', 'Primary Energy|Gas', 'Primary Energy|Oil'],
    'secondary_fossil': ['Secondary Energy|Electricity|Coal', 'Secondary Energy|Electricity|Gas', 'Secondary Energy|Electricity|Oil'],
    'secondary_renewable': ['Secondary Energy|Electricity|Biomass', 'Secondary Energy|Electricity|Geothermal',
                            'Secondary Energy|Electricity|Solar|CSP', 'Secondary Energy|Electricity|Nuclear',
                            'Secondary Energy|Electricity|Solar|PV', 'Secondary Energy|Electricity|Wind', 'Secondary Energy|Electricity|Hydro']
}

max_percentages = {
    'primary_fossil': 0,
    'secondary_fossil': 0,
    'secondary_renewable': 100
}

portfolio.rename(columns={'Regione': 'REGION'}, inplace=True)

lgd_lookup = portfolio.set_index('Titolo')['LGD_j'].to_dict()


def process_simulation(sim):
    local_results = []
    for _, row in portfolio.iterrows():
        bond = row['Titolo']
        region = row['REGION']

        bond_sectors = []

        for sector, percentage in max_percentages.items():
            if percentage > 0:
                available_sectors = sectors_data[sector]
                selected_subsectors = random.sample(available_sectors, min(max_subsectors_per_sector, len(available_sectors)))
                bond_sectors.extend(selected_subsectors)

        lgd_j = lgd_lookup.get(bond, None)

        for sector in bond_sectors:
            local_results.append({
                'simulation_num': sim,
                'Titolo': bond,
                'REGION': region,
                'sector': sector,
                'LGD_j': lgd_j,
            })
    return local_results


results = Parallel(n_jobs=-1)(
    delayed(process_simulation)(sim) for sim in tqdm(range(1, num_simulations + 1), desc="Processing simulations")
)

results_df = pd.DataFrame([item for sublist in results for item in sublist])



def generate_integer_weights(group):
    """
    Generates weights as normalized integers with a sum exactly equal to 100.

    Parameters:
    - group (DataFrame): Group of rows to associate the weights with.

    Returns:
    - group (DataFrame): Group with a new 'weight' column containing the integer weights.
    """
    num_rows = len(group)
    weights = np.random.randint(1, 100, size=num_rows)

    weights = np.round(weights / weights.sum() * 100).astype(int)

    difference = 100 - np.sum(weights)
    weights[0] += difference  # Corrects the difference in the first element

    weights = np.clip(weights, 0, None)

    group['weight'] = weights
    return group

results_df = results_df.groupby(['simulation_num', 'Titolo']).apply(generate_integer_weights).reset_index(drop=True)

print(results_df)

years = list(range(2025, 2055, 5))

new_table = []

def process_row(row, shock_data):
    sim = row['simulation_num']
    bond = row['Titolo']  
    region = row['REGION']
    sector = row['sector']
    weight = row['weight']
    lgd_j = row['LGD_j']

    shock_filter = shock_data[(shock_data['REGION'] == region) & (shock_data['VARIABLE'] == sector)]

    if not shock_filter.empty:
        new_row_list = []
        for _, shock_row in shock_filter.iterrows():
            model = shock_row['MODEL']
            scenario = shock_row['SCENARIO']

            new_row = {
                'simulation_num': sim,
                'Titolo': bond,
                'REGION': region,
                'sector': sector,
                'model': model,
                'scenario': scenario,
                'weight': weight,
                'LGD_j': lgd_j
            }

            for year in range(2025, 2055, 5):
                shock_value = shock_row[str(year)]
                new_row[f'shock_{year}'] = (shock_value * weight) / 100

            new_row_list.append(new_row)
        return new_row_list
    return []

num_cores = -1  
results = Parallel(n_jobs=num_cores, backend="loky")(
    delayed(process_row)(row, shock_data) for _, row in tqdm(results_df.iterrows(), total=len(results_df), desc="Parallel processing")
)

new_table = [row for sublist in results for row in sublist]

new_table_df = pd.DataFrame(new_table)

ordered_columns = ['simulation_num', 'Titolo', 'REGION', 'sector', 'model', 'scenario', 'weight'] + \
                      [f'shock_{year}' for year in range(2025, 2055, 5)]

new_table_df = new_table_df[ordered_columns]

print(new_table_df.head())


weighted_table = new_table_df.copy()

for year in range(2025, 2055, 5):
    shock_column = f'shock_{year}'  
    weighted_column = f'weighted_shock_{year}'  

    weighted_table[weighted_column] = (weighted_table[shock_column] * weighted_table['weight']) / 100

final_columns = ['simulation_num', 'Titolo', 'REGION', 'sector', 'model', 'scenario'] + \
                  [f'weighted_shock_{year}' for year in range(2025, 2055, 5)]
weighted_table = weighted_table[final_columns]

print(weighted_table.head())

del new_table_df
gc.collect()

years = range(2025, 2055, 5)

weighted_shock_columns = [f'weighted_shock_{year}' for year in years]

total_shocks = weighted_table.groupby(
    ['simulation_num', 'Titolo', 'model', 'scenario', 'REGION'],
    as_index=False
)[weighted_shock_columns].sum()

total_shocks.rename(columns={f'weighted_shock_{year}': f'shock_tot_{year}' for year in years}, inplace=True)

total_shocks = total_shocks.merge(
    max_shock_by_region[['REGION', 'MODEL', 'SCENARIO'] + [f'Max_Shock_{year}' for year in years]],
    left_on=['REGION', 'model', 'scenario'],
    right_on=['REGION', 'MODEL', 'SCENARIO'],
    how='left'
)

total_shocks.rename(columns={f'Max_Shock_{year}': f'max_shock_{year}' for year in years}, inplace=True)

total_shocks.drop(columns=['MODEL', 'SCENARIO'], inplace=True)

print(total_shocks.head())

total_shocks = total_shocks.merge(
    portfolio[['Titolo', 'LGD_j']],  # Take only Titolo and LGD_j from portfolio
    on='Titolo',                       # Merge key
    how='left'                         # Keep all rows of total_shocks
)

print(total_shocks.head())

print("Dimensions of total_shocks after merge:", total_shocks.shape)

del weighted_table
gc.collect()

years = range(2025, 2055, 5)

delta_v_results = []

for _, row in total_shocks.iterrows():
    new_row = {
        'simulation_num': row['simulation_num'],
        'bond_num': row['Titolo'],
        'model': row['model'],
        'scenario': row['scenario']
    }

    for year in years:
        new_row[f'delta_v_{year}'] = - (row['LGD_j'] * row[f'shock_tot_{year}']) / (row[f'max_shock_{year}'] + 1)

    delta_v_results.append(new_row)

delta_v_df = pd.DataFrame(delta_v_results)

print(delta_v_df.head())

years = range(2025, 2055, 5)
delta_v_cols = [f'delta_v_{year}' for year in years] 

aggregated_losses = delta_v_df.groupby(['simulation_num', 'model', 'scenario'], as_index=False)[delta_v_cols].sum()

rename_dict = {f'delta_v_{year}': f'sum_delta_v_{year}' for year in years}
aggregated_losses.rename(columns=rename_dict, inplace=True)

print(aggregated_losses.head())

years = range(2025, 2055, 5)
loss_cols = [f'sum_delta_v_{year}' for year in years]  # Loss columns

percentile_results_list = []

for year, loss_col in zip(years, loss_cols):
    percentile_col = f'percentile_99.5_{year}'

    if loss_col in aggregated_losses.columns:
        temp_percentile = aggregated_losses.groupby(['model', 'scenario'], as_index=False)[loss_col].quantile(0.995)

        temp_percentile.rename(columns={loss_col: percentile_col}, inplace=True)

        percentile_results_list.append(temp_percentile)
    else:
        print(f"Column {loss_col} not found in aggregated_losses.")

percentile_results = percentile_results_list[0]
for i in range(1, len(percentile_results_list)):
    percentile_results = pd.merge(percentile_results, percentile_results_list[i], on=['model', 'scenario'], how='inner')

print(percentile_results)

years = range(2025, 2055, 5)
risk_free_rate = 0.03  

adjusted_table = percentile_results.copy()

for year in years:
    T = year - 2025 
    discount = np.exp(-risk_free_rate * T)  

    percentile_col = f'percentile_99.5_{year}'
    adjusted_col = f'adjusted_{year}'

    if percentile_col in adjusted_table.columns:
        adjusted_table[adjusted_col] = adjusted_table[percentile_col] * discount
    else:
        print(f"Column {percentile_col} not found in adjusted_table.")

columns_to_remove = [f'percentile_99.5_{year}' for year in years]
adjusted_table.drop(columns=columns_to_remove, inplace=True, errors='ignore')

print(adjusted_table.head())

output_path = '/users/lorenzovancadsand/downloads/Basis_Africa_ROIND.xlsx'
adjusted_table.to_excel(output_path, index=False)

end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")

Table of associated values:
  Rating  p_mean      w
0      A    0.06  1.011
1    BBB    0.20  0.836
2     BB    1.25  0.602
3      B    6.25  0.415
4    CCC   17.50  0.295
Table with mu_x values:
  Rating  p_mean      w       mu_x
0      A    0.06  1.011   0.363348
1    BBB    0.20  0.836   1.018822
2     BB    1.25  0.602   4.760248
3      B    6.25  0.415  17.378554
4    CCC   17.50  0.295  37.119724
99° Percentile: 11.990692001306325
Portfolio simulations:
    Titolo     Attivi    Passivi  LGD Regione
0        1  56.410642  51.768305  0.6    ASIA
1        2  61.680751  50.518245  0.6    ASIA
2        3  58.223936  47.087378  0.6    ASIA
3        4  60.200186  52.684805  0.6    ASIA
4        5  58.910942  50.868502  0.6    ASIA
..     ...        ...        ...  ...     ...
95      96  60.102581  50.003287  0.6    ASIA
96      97  61.652126  49.799642  0.6    ASIA
97      98  63.053953  42.967727  0.6    ASIA
98      99  60.933829  51.643804  0.6    ASIA
99     100  59.580573  43.7988

Processing simulations: 100%|██████████| 1000/1000 [00:00<00:00, 4189.52it/s]


        simulation_num  Titolo REGION  \
0                    1       1   ASIA   
1                    1       1   ASIA   
2                    1       1   ASIA   
3                    1       2   ASIA   
4                    1       2   ASIA   
...                ...     ...    ...   
299995            1000      99   ASIA   
299996            1000      99   ASIA   
299997            1000     100   ASIA   
299998            1000     100   ASIA   
299999            1000     100   ASIA   

                                         sector  LGD_j  weight  
0          Secondary Energy|Electricity|Biomass  0.002      45  
1            Secondary Energy|Electricity|Hydro  0.002      48  
2             Secondary Energy|Electricity|Wind  0.002       7  
3          Secondary Energy|Electricity|Nuclear  0.195      23  
4            Secondary Energy|Electricity|Hydro  0.195      23  
...                                         ...    ...     ...  
299995       Secondary Energy|Electricity|Hydro  0.0

Parallel processing: 100%|██████████| 300000/300000 [00:21<00:00, 14008.20it/s]


   simulation_num  Titolo REGION                                sector  \
0               1       1   ASIA  Secondary Energy|Electricity|Biomass   
1               1       1   ASIA    Secondary Energy|Electricity|Hydro   
2               1       1   ASIA     Secondary Energy|Electricity|Wind   
3               1       2   ASIA  Secondary Energy|Electricity|Nuclear   
4               1       2   ASIA    Secondary Energy|Electricity|Hydro   

    model               scenario  weight  shock_2025  shock_2030  shock_2035  \
0  REMIND  LIMITSOilIndependence      45     0.90000     0.51705     0.23805   
1  REMIND  LIMITSOilIndependence      48     0.00192     0.00240     0.00288   
2  REMIND  LIMITSOilIndependence       7     0.00287     0.00329     0.00357   
3  REMIND  LIMITSOilIndependence      23     0.00437     0.01242     0.02231   
4  REMIND  LIMITSOilIndependence      23     0.00092     0.00115     0.00138   

   shock_2040  shock_2045  shock_2050  
0     0.11340     0.07335     0.05